<a href="https://colab.research.google.com/github/alexgabriel28/neural_style_transfer/blob/main/Neural_Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following notebook is mostly adapted from the great book "Deep Learning with Python", by Francois Chollet

In [ ]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import random
from PIL import Image

from google.colab import drive

In [ ]:
drive.mount("/content/drive") 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def preprocess_img(img_path):
  img = tf.keras.utils.load_img(img_path, target_size = (img_height, img_width))
  img = tf.keras.utils.img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  img = keras.applications.vgg19.preprocess_input(img)
  return img

def deprocess_img(img):
  img = img.reshape((img_height, img_width, 3))
  img[:,:,0] += 103.939
  img[:,:,1] += 116.779
  img[:,:,2] += 123.68
  img = img[:,:,::-1]
  img = np.clip(img, 0, 255).astype("uint8")
  return img

In [ ]:
model = keras.applications.vgg19.VGG19(weights = "imagenet", include_top = False)
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = keras.Model(inputs=model.inputs, outputs = outputs_dict)

In [ ]:
def content_loss(base_img, combination_img):
  return tf.reduce_sum(tf.square(combination_img - base_img))

In [ ]:
def gram_matrix(x):
  x = tf.transpose(x, (2,0,1))
  features = tf.reshape(x, (tf.shape(x)[0], -1))
  gram = tf.matmul(features, tf.transpose(features))
  return gram

def style_loss(style_img, combination_img):
  S = gram_matrix(style_img)
  C = gram_matrix(combination_img)
  channels = 3
  size = img_height * img_width
  return tf.reduce_sum(tf.square(S - C)) / (4.0 * ( channels ** 2) * (size ** 2))

In [ ]:
def total_variation_loss(x):
  a = tf.square(x[:, :img_height-1, :img_width-1, :] - x[:, 1:, :img_width-1, :])
  b = tf.square(x[:, :img_height-1, :img_width-1, :] - x[:, :img_height-1, 1:, :])
  return tf.reduce_sum(tf.pow(a+b, 1.25))

In [ ]:
style_layer_names = ["block1_conv1", "block2_conv1", "block3_conv1", "block4_conv1", "block5_conv1"]
content_layer_name = "block5_conv2"

total_variation_weight = 1e-6
style_weight = 1e-5
content_weight = 7.5e-8

In [ ]:
def compute_loss(combination_image, base_image, style_reference_image):
  input_tensor = tf.concat([base_image, style_reference_image, combination_image], axis = 0)
  features = feature_extractor(input_tensor)
  loss = tf.zeros(shape = ())
  layer_features = features[content_layer_name]
  base_image_features = layer_features[0, :, :, :]
  combination_features = layer_features[2, :, :, :]
  loss = loss + content_weight*content_loss(base_image_features, combination_features)
 
  for layer_name in style_layer_names:
    layer_features = features[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    style_loss_value = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layer_names)) * style_loss_value

  loss += total_variation_weight * total_variation_loss(combination_image)
  return loss

In [ ]:
paths = ["/content/marilyn.jpg", "/content/drive/MyDrive/Monet_GAN_Images/Generated/luana_alex_popart_2210.png"]
new_width  = 900
new_height = 1600
img_width = 900
img_height = 1600

i = 0
for path in paths:
  img = Image.open(path) # image extension *.png,*.jpg
  img = img.resize((new_height, new_width), Image.ANTIALIAS).rotate(0)
  img.save(f"img_{i}.jpg")
  i += 1

# Add the paths to the images here
base_image_path = "/content/img_1.jpg"
style_reference_image_path = "/content/img_0.jpg"

In [ ]:
@tf.function
def compute_loss_and_grads(
    combination_image, 
    base_image, 
    style_reference_image,
    root_dir,
    ):
  with tf.GradientTape() as tape:
    loss = compute_loss(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads
  
optimizer = tf.keras.optimizers.SGD(tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate = 150.0, decay_steps = 1000, decay_rate = 0.98))

base_image = preprocess_img(base_image_path)
style_reference_image = preprocess_img(style_reference_image_path)
combination_image = tf.Variable(preprocess_img(base_image_path))

iterations = 4005
for i in range(1, iterations + 1):
  loss, grads = compute_loss_and_grads(combination_image, base_image, style_reference_image)
  optimizer.apply_gradients([(grads, combination_image)])
  img = deprocess_img(combination_image.numpy())   
  if i % 10 == 0:
    path = path + str(i) + ".png"
    tf.keras.utils.save_img(path, img)
    print(f"Iterations {i}: loss = {loss: .2f}")

In [ ]:
img_height  = 256
img_width = 256

photo_paths = []
monet_paths = []

for file in os.listdir("/content/drive/MyDrive/Monet_GAN_Images/Photos/"):
  photo_paths.append(file)

for file in os.listdir("/content/drive/MyDrive/Monet_GAN_Images/Monet/"):
  monet_paths.append(file)

In [ ]:
img_height  = 256
img_width = 256

In [ ]:
for photo in photo_paths:
  base_image_path = os.path.join("/content/drive/MyDrive/Monet_GAN_Images/Photos/", photo)
  monet_choice = random.choice(monet_paths)
  style_reference_image_path = "/content/drive/MyDrive/Monet_GAN_Images/Monet/" + monet_choice
  
  photo_name = photo.split(".")[0]
  monet_name = monet_choice.split(".")[0]

  @tf.function
  def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
      loss = compute_loss(combination_image, base_image, style_reference_image)
      grads = tape.gradient(loss, combination_image)
      return loss, grads
    
  optimizer = tf.keras.optimizers.SGD(tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate = 100.0, decay_steps = 200, decay_rate = 0.96))

  base_image = preprocess_img(base_image_path)
  style_reference_image = preprocess_img(style_reference_image_path)
  combination_image = tf.Variable(preprocess_img(base_image_path))

  iterations = 2000
  for i in range(1, iterations + 1):
    loss, grads = compute_loss_and_grads(combination_image, base_image, style_reference_image)
    optimizer.apply_gradients([(grads, combination_image)])
    if i % 100 == 0:
      print(f"Iterations {i}: loss = {loss: .2f}")
    if i % 2000 == 0:
      img = deprocess_img(combination_image.numpy())
      path = f"/content/drive/MyDrive/Monet_GAN_Images/Generated/monet_style_transfer_{photo_name}_{monet_name}_{i%1000}.png"
      tf.keras.utils.save_img(path, img)